# 라이브러리

In [178]:
options(warn =-1, repr.plot.width=15, repr.plot.height=9)
# ## 패키지 설치
library(dplyr)
library(stringr)
library(ggplot2)
library(gridExtra)
library(scales)
library(randomForest)
library(lubridate)
library(caret)
library(e1071)
library(nnet)
library(class)


# 전처리

In [179]:
### .csv 불러오기
data <- read.csv("./data/refined_data_filtered_major5_subset.csv")

In [180]:
str(data)
unique(data$취업직종대분류)

'data.frame':	24979 obs. of  10 variables:
 $ 취업일자         : chr  "2020-01-01" "2020-01-01" "2020-01-01" "2020-01-01" ...
 $ 연령             : int  65 61 53 64 53 60 63 40 56 62 ...
 $ 장애유형         : chr  "시각장애" "시각장애" "시각장애" "시각장애" ...
 $ 중증여부         : chr  "중증" "중증" "경증" "중증" ...
 $ 근무지역         : chr  "경북 경산시" "경북 고령군" "경북 의성군" "경북 성주군" ...
 $ 취업직종대분류   : chr  "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" "경영·행정·사무직" ...
 $ 연령대           : int  5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: chr  "시각장애-중증" "시각장애-중증" "시각장애-경증" "시각장애-중증" ...
 $ 월               : int  1 1 1 1 1 1 2 3 7 1 ...
 $ 근무지역분류     : chr  "경북" "경북" "경북" "경북" ...


[1] "경영·행정·사무직"                       
 [2] "보건·의료직"                            
 [3] "전기·전자 설치·정비·생산직"             
 [4] "사회복지·종교직"                        
 [5] "청소 및 기타 개인서비스직"              
 [6] "돌봄 서비스직(간병·육아)"               
 [7] "음식 서비스직"                          
 [8] "스포츠·레크리에이션직"                  
 [9] "경호·경비직"                            
[10] "운전·운송직"                            
[11] "제조 단순직"                            
[12] "예술·디자인·방송직"                     
[13] "인쇄·목재·공예 및 기타 설치·정비·생산직"
[14] "교육직"                                 
[15] "영업·판매직"                            
[16] "섬유·의복 생산직"                       
[17] "식품 가공·생산직"                       
[18] "기계 설치·정비·생산직"                  
[19] "건설·채굴직"                            
[20] "관리직(임원·부서장)"                    
[21] "미용·예식 서비스직"

In [181]:
data$근무지역분류 <- as.character(data$근무지역분류)
data$근무지역분류 <- factor(data$근무지역분류)
data$장애유형 <- as.character(data$장애유형)
data$장애유형 <- factor(data$장애유형)
data$중증여부 <- as.character(data$중증여부)
data$중증여부 <- factor(data$중증여부)
data$근무지역 <- as.character(data$근무지역)
data$근무지역 <- factor(data$근무지역)
data$취업직종대분류 <- as.character(data$취업직종대분류)
data$취업직종대분류 <- factor(data$취업직종대분류)
data$연령대 <- as.character(data$연령대)
data$연령대 <- factor(data$연령대)
data$장애유형_중증여부 <- as.character(data$장애유형_중증여부)
data$장애유형_중증여부 <- factor(data$장애유형_중증여부)
data$월 <- as.character(data$월)
data$월 <- factor(data$월)

In [182]:
# 불필요한 컬럼 제거

data_filtered <- subset(data, select = -c(취업일자,연령,장애유형,중증여부,근무지역))
sum(is.na(data_filtered))

[1] 0

# lm 확인

In [183]:
data_for_lm<-data_filtered
data_for_lm$target<-ifelse(data_for_lm$취업직종대분류 == "보건·의료직", 1, 0)
data_for_lm <- subset(data_for_lm, select = -취업직종대분류)
str(data_for_lm)

'data.frame':	24979 obs. of  5 variables:
 $ 연령대           : Factor w/ 6 levels "1","2","3","4",..: 5 5 4 5 4 5 5 3 4 5 ...
 $ 장애유형_중증여부: Factor w/ 10 levels "시각장애-경증",..: 2 2 1 2 1 2 2 2 8 2 ...
 $ 월               : Factor w/ 12 levels "1","10","11",..: 1 1 1 1 1 1 5 6 10 1 ...
 $ 근무지역분류     : Factor w/ 17 levels "강원","경기",..: 4 4 4 4 4 4 4 4 1 1 ...
 $ target           : num  0 0 0 0 0 0 0 1 1 1 ...


In [184]:
lmthis<-lm(target~.,data_for_lm)

In [185]:
summary(lmthis)


Call:
lm(formula = target ~ ., data = data_for_lm)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.76992 -0.03329 -0.01834 -0.00574  1.02205 

Coefficients:
                                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)                     0.0618232  0.0083443   7.409 1.31e-13 ***
연령대2                         0.0040244  0.0036409   1.105  0.26903    
연령대3                         0.0174472  0.0040323   4.327 1.52e-05 ***
연령대4                         0.0328284  0.0042711   7.686 1.57e-14 ***
연령대5                         0.0183749  0.0045309   4.055 5.02e-05 ***
연령대6                         0.0092394  0.0065701   1.406  0.15965    
장애유형_중증여부시각장애-중증  0.6300747  0.0067690  93.082  < 2e-16 ***
장애유형_중증여부정신장애-경증 -0.0561417  0.0582862  -0.963  0.33545    
장애유형_중증여부정신장애-중증 -0.0437571  0.0066044  -6.625 3.53e-11 ***
장애유형_중증여부지적장애-경증 -0.0522758  0.0371136  -1.409  0.15899    
장애유형_중증여부지적장애-중증 -0.0300708  0.0055392  -5.429 5.73e-08 ***
장애유형_중증여부지체장애-경증 -0.044583

In [186]:
# 연령대
data_filtered$age_20 <- data_filtered$연령대 == 1
data_filtered$age_30 <- data_filtered$연령대 == 2
data_filtered$age_40 <- data_filtered$연령대 == 3
data_filtered$age_50 <- data_filtered$연령대 == 4
data_filtered$age_60 <- data_filtered$연령대 == 5
data_filtered$age_70 <- data_filtered$연령대 == 6

In [187]:
# 장애유형_중증여부 바꾸기
data_filtered$시각장애_중증 <- data_filtered$장애유형_중증여부 == "시각장애-중증"
data_filtered$시각장애_경증 <- data_filtered$장애유형_중증여부 == "시각장애-경증"

data_filtered$지체장애_중증 <- data_filtered$장애유형_중증여부 == "지체장애-중증"
data_filtered$지체장애_경증 <- data_filtered$장애유형_중증여부 == "지체장애-경증"

data_filtered$지적장애_경증 <- data_filtered$장애유형_중증여부 == "지적장애-경증"
data_filtered$지적장애_중증 <- data_filtered$장애유형_중증여부 == "지적장애-중증"

data_filtered$청각장애_경증 <- data_filtered$장애유형_중증여부 == "청각장애-경증"
data_filtered$청각장애_중증 <- data_filtered$장애유형_중증여부 == "청각장애-중증"

data_filtered$정신장애_경증 <- data_filtered$장애유형_중증여부 == "정신장애-경증"
data_filtered$정신장애_중증 <- data_filtered$장애유형_중증여부 == "정신장애-중증"

In [188]:
#   근무지역 분류 바꾸기
data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

data_filtered$강원 <- data_filtered$근무지역분류 == "강원"

data_filtered$전남 <- data_filtered$근무지역분류 == "전남"

data_filtered$충북 <- data_filtered$근무지역분류 == "충북"

data_filtered$경기 <- data_filtered$근무지역분류 == "경기"

data_filtered$인천 <- data_filtered$근무지역분류 == "인천"

data_filtered$서울 <- data_filtered$근무지역분류 == "서울"

data_filtered$울산 <- data_filtered$근무지역분류 == "울산"

data_filtered$대전 <- data_filtered$근무지역분류 == "대전"

data_filtered$부산 <- data_filtered$근무지역분류 == "부산"

data_filtered$전북 <- data_filtered$근무지역분류 == "전북"

data_filtered$광주 <- data_filtered$근무지역분류 == "광주"

data_filtered$충남 <- data_filtered$근무지역분류 == "충남"

data_filtered$대구 <- data_filtered$근무지역분류 == "대구"

data_filtered$경남 <- data_filtered$근무지역분류 == "경남"

data_filtered$제주 <- data_filtered$근무지역분류 == "제주"

data_filtered$세종 <- data_filtered$근무지역분류 == "세종"

data_filtered$경북 <- data_filtered$근무지역분류 == "경북"

In [189]:
# 월
data_filtered$Jan <- data_filtered$월  == 01
data_filtered$Feb <- data_filtered$월  == 02
data_filtered$Mar <- data_filtered$월  == 03
data_filtered$Apr <- data_filtered$월  == 04
data_filtered$May <- data_filtered$월  == 05
data_filtered$Jun <- data_filtered$월  == 06
data_filtered$Jul <- data_filtered$월  == 07
data_filtered$Aug <- data_filtered$월  == 08
data_filtered$Sep <- data_filtered$월  == 09
data_filtered$Oct <- data_filtered$월  == 10
data_filtered$Nov <- data_filtered$월  == 11
data_filtered$Dec <- data_filtered$월  == 12


In [190]:
# 불필요한 컬럼 제거
data_filtered_subset <- subset(data_filtered, select = -c(연령대,장애유형_중증여부,월,근무지역분류))

# 변동사항

In [191]:
data_filtered_subset$target<-ifelse(data_filtered_subset$취업직종대분류 == "보건·의료직", 1, 0)

In [192]:
data_filtered_subset$target <- factor(data_filtered_subset$target)

In [193]:
data_filtered_subset <- subset(data_filtered_subset, select = -취업직종대분류)

# 시험준비

In [194]:
samp <- createDataPartition(data_filtered_subset$target, p=0.7, list=F)

#   학습용
data.tr <- data_filtered_subset[samp,]

#   검증용
data.test <- data_filtered_subset[-samp,]

In [195]:
#   KNN을 위한 Training Data 준비
data.training <- subset(data.tr, select = -target)
data.training.label <- data.tr$target

In [196]:
#   문제 및 정답 만들기
x   <- subset(data.test, select = -target)
y   <- data.test$target


# 학습시키기

In [197]:
#  랜덤포레스트 학습 시키기
rf <- randomForest(target ~., data.tr, ntree=100)

In [198]:
# SVM 학습시키기
modelsvm <- svm(target~.,data.tr)

In [199]:
# KNN 학습시키기 3,5,7,
knnmodel <- knn(train=data.training, test=x,cl=data.training.label,k=3)

ERROR: Error in knn(train = data.training, test = x, cl = data.training.label, : too many ties in knn


In [ ]:
# nnet 학습시키기
modelnnet <- nnet(target~.,data.tr,size =10)

# weights:  471
initial  value 12401.104003 
iter  10 value 4179.821072
iter  20 value 2468.966493
iter  30 value 2381.150397
iter  40 value 2347.844123
iter  50 value 2282.643019
iter  60 value 2256.649756
iter  70 value 2248.833207
iter  80 value 2245.582130
iter  90 value 2239.349315
iter 100 value 2232.435391
final  value 2232.435391 
stopped after 100 iterations


# 검증하기

In [ ]:
#  Random Forest 검증하기
predrf <- predict(rf, x)

In [ ]:
#  SVM 검증하기
predmodelsvm <- predict(modelsvm, x)

In [ ]:
# KNN 검증하기
mean(knnmodel==y) 

[1] 0.9595623

In [ ]:
# nnet 검증하기
prednnet = predict(modelnnet, x,type = "class")    # class : 객관식, raw : 주관식

# 예측력 확인

In [ ]:
#  RF 예측력
mean(predrf == y)

[1] 0.9630322

In [ ]:
#  SVM 예측력
mean(predmodelsvm == y)

[1] 0.9602295

In [ ]:
#  KNN 예측력
mean(knnmodel == y)

[1] 0.9595623

In [ ]:
#  nnet 예측력
mean(prednnet == y)

[1] 0.9642333

# 오분류표

In [ ]:
#  Random Forest 오분류표
table(predrf,y)

      y
predrf    0    1
     0 6930  196
     1   81  286

In [ ]:
#  SVM 오분류표
table(predmodelsvm,y)

            y
predmodelsvm    0    1
           0 6878  165
           1  133  317

In [ ]:
#  KNN 오분류표
table(y, knnmodel)

   knnmodel
y      0    1
  0 6938   73
  1  230  252

In [ ]:
# nnet 오분류표
table(prednnet,y)

        y
prednnet    0    1
       0 6933  190
       1   78  292

## 결과는 X 못쓴다.